In [1]:
# !apt-get update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
# !tar xf spark-2.3.1-bin-hadoop2.7.tgz
# !pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReadwriteVal").getOrCreate() 
spark

fifa19.csv   spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data  spark-2.3.1-bin-hadoop2.7.tgz  youtubevideos.csv


In [3]:
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
cores

1

In [4]:
airbnb = spark.read.csv('sample_data/nyc_air_bnb.csv',inferSchema=True,header=True)

In [5]:
airbnb.limit(4).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194


In [6]:
print(airbnb.printSchema())

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)

None


# **Casting**

In [7]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = airbnb.withColumn("price", airbnb["price"].cast(IntegerType())) \
        .withColumn("minimum_nights", airbnb["minimum_nights"].cast(IntegerType())) \
        .withColumn("number_of_reviews", airbnb["number_of_reviews"].cast(IntegerType())) \
        .withColumn("reviews_per_month", airbnb["reviews_per_month"].cast(IntegerType())) \
        .withColumn("calculated_host_listings_count", airbnb["calculated_host_listings_count"].cast(IntegerType()))
#QA
print(df.printSchema())
df.limit(5).toPandas()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.0,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.0,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.0,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.0,1,0


In [8]:
print(df.printSchema())

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None


# **GroupBy & Aggregation**

In [9]:
df.groupBy("neighbourhood_group").count().show(7)

+-------------------+-----+
|neighbourhood_group|count|
+-------------------+-----+
|         Douglaston|    1|
|             Queens| 5630|
|              Nadia|    1|
|            Midtown|    4|
|    Jackson Heights|    2|
|     Hell's Kitchen|    7|
|  Greenwich Village|    2|
+-------------------+-----+
only showing top 7 rows



In [10]:
df.groupBy("neighbourhood_group").mean("price").show(5)

+-------------------+-----------------+
|neighbourhood_group|       avg(price)|
+-------------------+-----------------+
|         Douglaston|              1.0|
|             Queens|99.57690941385435|
|              Nadia|             null|
|            Midtown|              9.0|
|    Jackson Heights|             16.0|
+-------------------+-----------------+
only showing top 5 rows



In [11]:
df.groupBy("neighbourhood").agg({'price':'mean'}).show(5)

+-------------+----------+
|neighbourhood|avg(price)|
+-------------+----------+
|       Corona| 59.171875|
| Richmondtown|      78.0|
| Prince's Bay|     409.5|
|  Westerleigh|      71.5|
|   Mill Basin|    179.75|
+-------------+----------+
only showing top 5 rows



In [12]:
df.groupBy("neighbourhood").agg(min(df.price).alias("Min Price"),max(df.price).alias("Max Price")).show(5)

+-------------+---------+---------+
|neighbourhood|Min Price|Max Price|
+-------------+---------+---------+
|       Corona|       23|      359|
| Richmondtown|       78|       78|
| Prince's Bay|       85|     1250|
|  Westerleigh|       40|      103|
|   Mill Basin|       85|      299|
+-------------+---------+---------+
only showing top 5 rows



In [13]:
summary = df.summary("count", "min", "25%", "75%", "max")
summary.toPandas()

,summary,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,count,49079,49047,48894,48873,48894,48894,48894,48894,48894,48887,48891,48738,38845,38858,48891,48737
1,min,"12 mins Manhattan""",1 Bed Apt in Utopic Williamsburg,"Heart of Greenwich Village""","very clean studio app""",194716858,2,-73.72247,-73.71299,-73.90783,-74,0,0,-73.94134,0,0,0
2,25%,9470677.0,2.4544724E7,7798275.0,475.0,1.94716858E8,40.68771,40.68982,-73.9831,56.0,69,1,1,0.76,0,1,0
3,75%,2.9150743E7,1.74786681E8,1.07434423E8,3.3499491E7,1.97400421E8,40.78304,40.76299,-73.93638,145.0,175,5,23,3.24,2,2,226
4,max,"獨一無二的紐約閣樓""","ﾏﾝﾊｯﾀﾝ､駅から徒歩4分でどこに行くのにも便利な場所!女性の方希望,ｷﾚｲなお部屋｡",呈刚,현선,Woodside,Woodside,West Village,Shared room,Shared room,10000,1250,629,9.66,58,365,365


In [14]:
limit_summary = df.select("price","minimum_nights","number_of_reviews").summary("count","min","max")
limit_summary.toPandas()

,summary,price,minimum_nights,number_of_reviews
0,count,48887,48891,48738
1,min,-74,0,0
2,max,10000,1250,629


# **Without GroupBy**

In [15]:
df.agg(min(df.price).alias("Min Price"),max(df.price).alias("Max Price")).show()

+---------+---------+
|Min Price|Max Price|
+---------+---------+
|      -74|    10000|
+---------+---------+



In [16]:
df.select(countDistinct("neighbourhood_group").alias('CountD'),avg('price'),stddev("price")).show()

+------+------------------+------------------+
|CountD|        avg(price)|stddev_samp(price)|
+------+------------------+------------------+
|    77|152.22298361527604| 238.5414668883948|
+------+------------------+------------------+



In [17]:
df.agg({'number_of_reviews':'max'}).withColumnRenamed("max(number_of_reviews)", "Max Reviews").show()

+-----------+
|Max Reviews|
+-----------+
|        629|
+-----------+



In [18]:
df.groupBy('neighbourhood_group').count().orderBy('count', ascending = False).show(10)

+-------------------+-----+
|neighbourhood_group|count|
+-------------------+-----+
|          Manhattan|21594|
|           Brooklyn|20055|
|             Queens| 5630|
|              Bronx| 1080|
|      Staten Island|  370|
|               null|  185|
|             Harlem|   13|
| Bedford-Stuyvesant|    9|
|    Upper East Side|    7|
|           Elmhurst|    7|
+-------------------+-----+
only showing top 10 rows



# **Pivot**

In [19]:
df.groupBy("room_type").pivot("neighbourhood_group", ["Queens", "Brooklyn"]).count().show(10)

+-----------+------+--------+
|  room_type|Queens|Brooklyn|
+-----------+------+--------+
|         51|  null|    null|
|        205|  null|    null|
|         54|  null|    null|
|        200|  null|    null|
|        279|  null|    null|
|        138|  null|    null|
|         69|  null|    null|
|         42|  null|    null|
|Shared room|   198|     413|
|  -73.95777|  null|    null|
+-----------+------+--------+
only showing top 10 rows



In [20]:
df.filter("room_type='Shared room'").groupBy("room_type").pivot("neighbourhood_group", ["Queens", "Brooklyn"]).count().show(10)

+-----------+------+--------+
|  room_type|Queens|Brooklyn|
+-----------+------+--------+
|Shared room|   198|     413|
+-----------+------+--------+



# **Groupby Pivot Agg COMBINED**

In [21]:
df.groupBy("neighbourhood").pivot("neighbourhood_group", ["Queens", "Brooklyn"]).agg(min(df.price).alias("Min Price"),max(df.price).alias("Max Price")).show()

+-------------+----------------+----------------+------------------+------------------+
|neighbourhood|Queens_Min Price|Queens_Max Price|Brooklyn_Min Price|Brooklyn_Max Price|
+-------------+----------------+----------------+------------------+------------------+
|       Corona|              23|             359|              null|              null|
| Prince's Bay|            null|            null|              null|              null|
| Richmondtown|            null|            null|              null|              null|
|   Mill Basin|            null|            null|                85|               299|
|  Westerleigh|            null|            null|              null|              null|
|     40.76199|            null|            null|              null|              null|
| Civic Center|            null|            null|              null|              null|
|   Douglaston|              40|             178|              null|              null|
|     40.83166|            null|